# (Optional Lab) Nova Prompt Optimizer

## Introduction

The Nova Prompt Optimizer is a powerful tool that automatically improves your prompts for Amazon Nova models using your own datasets. This workshop demonstrates how to:

1. Transform manual prompt engineering into an efficient, data-driven process
2. Optimize prompts specifically tailored to your use case and data
3. Evaluate the performance improvements from optimization

By the end of this notebook, you'll understand how to leverage automated prompt optimization to unlock the full potential of Amazon Nova models for your specific applications.

<div class="alert alert-block alert-warning">
    <b>Optional Lab</b> 
    
    This notebook takes 15 mins to run. Recommend to treat it as an optional notebook for AWS hosted event.
</div>

## Section 1: Setup and Installation

We'll start by installing the Nova Prompt Optimizer SDK, which provides the tools needed to automatically optimize prompts based on your data.

In [ ]:
import sys
!{sys.executable} -m pip install nova-prompt-optimizer

## Section 2: Initialize the Input Adapters

The Nova Prompt Optimizer uses adapters to standardize inputs from different sources. These adapters help connect your data, prompts, and evaluation metrics into the optimization pipeline.

![adapters](nova_prompt_optimizer/docs/adapters.png)

### 2.1 Dataset Adapter

The Dataset Adapter converts your data into a standardized format for optimization and evaluation:

- **Input Columns**: Specify which fields from your data will be used as inputs to the model
- **Output Columns**: Specify which fields contain the expected outputs for comparison
- **Train/Test Split**: Divide your dataset for optimization and evaluation

In [ ]:
from amzn_nova_prompt_optimizer.core.input_adapters.dataset_adapter import JSONDatasetAdapter

# Define which columns in our dataset contain inputs and expected outputs
input_columns = {"input"}  # The field containing the user's query
output_columns = {"answer"}  # The field containing the expected model response

# Initialize the dataset adapter for our JSONL dataset
dataset_adapter = JSONDatasetAdapter(input_columns, output_columns)

# Load and process the dataset
dataset_adapter.adapt("nova_prompt_optimizer/data/FacilitySupportAnalyzer.jsonl")

# Split into training data (for optimization) and test data (for evaluation)
train_set, test_set = dataset_adapter.split(0.5)  # 50/50 split

### 2.2 Prompt Adapter

The Prompt Adapter standardizes your existing prompt template:

- **Prompt Variables**: Identify placeholders in your prompt that should be replaced with data from input columns
- **File Path**: Provide the path to your original prompt template
- **Adapt**: Process the prompt into a standardized format for optimization

In [ ]:
from amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter import TextPromptAdapter

# Define which variables in our prompt will be replaced with data from input columns
prompt_variables = input_columns

# Initialize the prompt adapter for a text prompt file
prompt_adapter = TextPromptAdapter()

# Load the original prompt template and specify which variables to replace
prompt_adapter.set_user_prompt(file_path="nova_prompt_optimizer/original_prompt/user_prompt_template.txt", variables=prompt_variables)

# Process the prompt into a standardized format
prompt_adapter.adapt()

### 2.3 Metric Adapter

The Metric Adapter defines how to evaluate prompt performance:

- **Custom Metrics**: Create evaluation metrics specific to your task
- **Apply Function**: Evaluate a single model response against the expected output
- **Batch Apply Function**: Evaluate multiple responses at once

For this example, we'll create a custom metric for the Facility Support Analyzer task that measures:
1. JSON validity
2. Correctness of categories
3. Accuracy of sentiment classification
4. Accuracy of urgency classification

In [ ]:
from amzn_nova_prompt_optimizer.core.input_adapters.metric_adapter import MetricAdapter
from typing import List, Any, Dict
import re
import json

class FacilitySupportAnalyzerMetric(MetricAdapter):
    def parse_json(self, input_string: str):
        """
        Attempts to parse the given string as JSON. If direct parsing fails,
        it tries to extract a JSON snippet from code blocks formatted as:
            ```json
            ... JSON content ...
            ```
        or any code block delimited by triple backticks and then parses that content.
        """
        try:
            return json.loads(input_string)
        except json.JSONDecodeError as err:
            error = err

        patterns = [
            re.compile(r"```json\s*(.*?)\s*```", re.DOTALL | re.IGNORECASE),
            re.compile(r"```(.*?)```", re.DOTALL)
        ]

        for pattern in patterns:
            match = pattern.search(input_string)
            if match:
                json_candidate = match.group(1).strip()
                try:
                    return json.loads(json_candidate)
                except json.JSONDecodeError:
                    continue

        raise error

    def _calculate_metrics(self, y_pred: Any, y_true: Any) -> Dict:
        strict_json = False
        result = {
            "is_valid_json": False,
            "correct_categories": 0.0,
            "correct_sentiment": False,
            "correct_urgency": False,
        }

        try:
            y_true = y_true if isinstance(y_true, dict) else (json.loads(y_true) if strict_json else self.parse_json(y_true))
            y_pred = y_pred if isinstance(y_pred, dict) else (json.loads(y_pred) if strict_json else self.parse_json(y_pred))
        except json.JSONDecodeError:
            result["total"] = 0
            return result  # Return result with is_valid_json = False
        else:
            result["is_valid_json"] = True

            categories_true = y_true.get("categories", {})
            categories_pred = y_pred.get("categories", {})

            if isinstance(categories_true, dict) and isinstance(categories_pred, dict):
                correct = sum(
                    categories_true.get(k, False) == categories_pred.get(k, False)
                    for k in categories_true
                )
                result["correct_categories"] = correct / len(categories_true) if categories_true else 0.0
            else:
                result["correct_categories"] = 0.0  # or raise an error if you prefer

            result["correct_sentiment"] = y_pred.get("sentiment", "") == y_true.get("sentiment", "")
            result["correct_urgency"] = y_pred.get("urgency", "") == y_true.get("urgency", "")

        # Compute overall metric score
        result["total"] = sum(
            float(result[k]) for k in ["correct_categories", "correct_sentiment", "correct_urgency"]
        ) / 3.0

        return result

    def apply(self, y_pred: Any, y_true: Any):
        return self._calculate_metrics(y_pred, y_true)

    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        evals = [self.apply(y_pred, y_true) for y_pred, y_true in zip(y_preds, y_trues)]
        float_keys = [k for k, v in evals[0].items() if isinstance(v, (int, float, bool))]
        return {k: sum(e[k] for e in evals) / len(evals) for k in float_keys}

metric_adapter = FacilitySupportAnalyzerMetric()

### 2.4 Inference Adapter

The Inference Adapter connects to the model service:

- **Backend**: Currently supports Amazon Bedrock
- **Region**: Specify which AWS region to use for inference
- **Configuration**: Set up the connection to the inference service

In [ ]:
from amzn_nova_prompt_optimizer.core.inference.adapter import BedrockInferenceAdapter

# Initialize the inference adapter to connect to Amazon Bedrock
# We're using us-west-2 region for this example
inference_adapter = BedrockInferenceAdapter(region_name="us-west-2")

## Section 3: Evaluate the Original Prompt

Before optimization, we'll establish a baseline by evaluating the original prompt's performance on our test dataset. This will help us measure the improvement from optimization.

The Evaluator:
- Takes our prompt, test data, metrics, and inference adapter
- Generates predictions using the original prompt
- Calculates evaluation metrics on these predictions

#### Base Model Evaluation

In [ ]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

# Initialize the evaluator with all our components
# - prompt_adapter: The prompt to evaluate
# - test_set: Data to run the evaluation on
# - metric_adapter: How to calculate performance metrics
# - inference_adapter: Connection to the model service
evaluator = Evaluator(prompt_adapter, test_set, metric_adapter, inference_adapter)

In [ ]:
# Run evaluation of the original prompt with Amazon Nova Lite
# This will generate predictions and calculate metrics
original_prompt_score = evaluator.aggregate_score(model_id="us.amazon.nova-lite-v1:0")

print(f"Original Prompt Evaluation Score = {original_prompt_score}")

## Section 4: Optimize the Prompt

Now we'll use the Nova Prompt Optimizer to automatically improve our prompt based on the training data.

### 4.1 Optimization Metric

First, we need to adapt our metric for the optimizer, which requires a single numerical score instead of multiple metrics:

In [ ]:
class FacilitySupportAnalyzerNovaPromptOptimizerMetric(FacilitySupportAnalyzerMetric):
    def apply(self, y_pred: Any, y_true: Any):
        """
        Returns a single numerical value for the optimizer to use.
        The optimizer needs a single score to maximize during optimization.
        
        Args:
            y_pred: The model's prediction
            y_true: The expected output
            
        Returns:
            float: A score between 0 and 1, with higher being better
        """
        # Calculate metrics and return the total score (average of all metrics)
        return self._calculate_metrics(y_pred, y_true)["total"]
        
    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        # Not used during optimization
        pass
    
# Create the metric adapter for optimization
nova_prompt_optimizer_metric_adapter = FacilitySupportAnalyzerNovaPromptOptimizerMetric()

### 4.2 Optimization Adapters

Next, we'll set up the optimization process. The Nova Prompt Optimizer takes:

- **Prompt Adapter**: The original prompt to optimize
- **Inference Adapter**: Connection to the model service
- **Dataset Adapter**: Training data to learn from
- **Metric Adapter**: How to evaluate prompt performance

### 4.3 Nova Prompt Optimizer

The Nova Prompt Optimizer uses a two-stage approach:

1. **Meta Prompting**: Analyzes your prompt to identify system instructions and user template patterns
2. **MIPROv2 Optimization**: Improves system instructions and adds few-shot examples based on your dataset

The optimizer can run in different modes based on your Nova model:
- **Lite mode**: Optimized for Nova Lite, faster optimization with fewer resources
- **Pro mode**: Optimized for Nova Pro, more thorough optimization that may take longer

In [ ]:
from amzn_nova_prompt_optimizer.core.optimizers import NovaPromptOptimizer

# Initialize the Nova Prompt Optimizer with our components
nova_prompt_optimizer = NovaPromptOptimizer(
    prompt_adapter=prompt_adapter,        # Original prompt to optimize
    inference_adapter=inference_adapter,  # Connection to model service
    dataset_adapter=train_set,            # Training data to learn from
    metric_adapter=nova_prompt_optimizer_metric_adapter  # How to evaluate performance
)

# Run the optimization process in "lite" mode for Nova Lite
# This will analyze the prompt, identify improvements, and generate few-shot examples
optimized_prompt_adapter = nova_prompt_optimizer.optimize(mode="lite")

### 4.4 Examining the Optimized Prompt

Let's examine what the optimizer has produced. First, the optimized system prompt:

In [ ]:
optimized_prompt_adapter.system_prompt

### 4.5 Optimized User Prompt

Now let's look at the optimized user prompt template:

In [ ]:
optimized_prompt_adapter.user_prompt

### 4.6 Saving the Optimized Prompt

Let's save the optimized prompt for future use:

In [ ]:
optimized_prompt_adapter.save("nova_prompt_optimizer/optimized_prompt/")

## Section 5: Evaluate the Optimized Prompt

Now let's measure the performance of our optimized prompt on the test dataset to see how much improvement we've gained:

In [ ]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

# Create a new evaluator for the optimized prompt
evaluator = Evaluator(
    optimized_prompt_adapter,  # Now using the optimized prompt
    test_set,                  # Same test data as before
    metric_adapter,            # Same evaluation metrics
    inference_adapter          # Same model service
)

In [ ]:
# Run evaluation of the optimized prompt with Amazon Nova Lite
nova_prompt_optimizer_eval_score = evaluator.aggregate_score(model_id="us.amazon.nova-lite-v1:0")

In [ ]:
# Print the score and compare it to the original prompt
print(f"Optimized Prompt Evaluation Score = {nova_prompt_optimizer_eval_score}")
print(f"Improvement: {nova_prompt_optimizer_eval_score['total'] - original_prompt_score['total']:.4f} ({(nova_prompt_optimizer_eval_score['total'] - original_prompt_score['total']) / original_prompt_score['total'] * 100:.2f}%)")

### 5.1 Saving Evaluation Results

Let's save the detailed evaluation results for analysis:

In [ ]:
evaluator.save("nova_prompt_optimizer/evals/nova_lite/nova_prompt_optimizer_eval.jsonl")

# Conclusion

In this workshop, we've explored how to use the Nova Prompt Optimizer to automatically improve prompt performance for Amazon Nova models. Let's summarize what we've learned and the benefits of this approach.

## Key Learnings

### 1. The Power of Automated Optimization
- **Data-Driven Improvements**: Rather than manual trial-and-error, we used our own dataset to guide prompt optimization
- **Systematic Approach**: The optimizer methodically analyzes and enhances prompts through meta-prompting and few-shot learning
- **Measurable Results**: We quantitatively measured performance gains between original and optimized prompts

### 2. Components of the Nova Prompt Optimizer
- **Dataset Adapter**: Standardized our dataset for use in optimization and evaluation
- **Prompt Adapter**: Processed our original prompt into a format suitable for optimization
- **Metric Adapter**: Provided custom evaluation metrics specific to our task
- **Inference Adapter**: Connected us to Amazon Nova models for testing
- **Optimization Process**: Combined meta-prompting and MIPROv2 techniques

### 3. Optimization Techniques Applied
- **System Prompt Refinement**: Improved the system instructions for better task understanding
- **Few-Shot Example Selection**: Automatically identified the most helpful examples from our data
- **Format Optimization**: Enhanced output formatting and structure
- **Task-Specific Guidance**: Added task-specific tips and clarifications

## Benefits for Production Applications

1. **Reduced Engineering Time**: Automates the time-consuming process of prompt engineering
2. **Consistent Performance**: Creates reliable, tested prompts for production use
3. **Adaptability**: Easily update optimized prompts as your data or requirements change
4. **Model Flexibility**: Works with different Amazon Nova models (Micro, Lite, Pro)
5. **Customization**: Optimizes for your specific data and task requirements

## Next Steps

As you apply the [Nova Prompt Optimizer](https://github.com/aws/nova-prompt-optimizer) to your own projects, consider:

1. **Expand Your Dataset**: Larger, more diverse datasets often yield better optimization results
2. **Test Different Metrics**: Create custom metrics that align closely with your business goals
3. **Compare Models**: Try optimizing for different Nova models to find the best performance/cost balance
4. **Periodic Re-optimization**: Update your prompts as your data or requirements evolve
5. **Integration**: Incorporate optimized prompts into your production applications